In [1]:
%reload_ext autoreload
%autoreload 2

import asyncio
import json
import nest_asyncio
import os
import sys
from dotenv import load_dotenv
import numpy as np
import pandas as pd

sys.path.append('../')
from lattereview.providers.openai_provider import OpenAIProvider
from lattereview.providers.ollama_provider import OllamaProvider
from lattereview.providers.litellm_provider import LiteLLMProvider
from lattereview.agents.scoring_reviewer import ScoringReviewer
from lattereview.review_workflow import ReviewWorkflow

Failed to update token costs. Using static costs.
/Users/pouria/Documents/Coding/LatteReview/venv/lib/python3.9/site-packages/tokencost/constants.py:69: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  logger.error("Failed to update token costs. Using static costs.")
/Users/pouria/Documents/Coding/LatteReview/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setting up the notebook

Loading environment variables:

In [2]:
# Load environment variables from .env file
load_dotenv('../.env')
print(os.getenv('OPENAI_API_KEY'))

# Enable asyncio in Jupyter
nest_asyncio.apply()

sk-cq_M0pNgHhCFnlDOCMnagYA1l2X7Yea5CL0ci5pZMNT3BlbkFJ0m0x9wm5M_EstX5SjLu_kdwGMDYjkUdviNPs4pe9cA


Loading a dummy dataset:

In [3]:
data = pd.read_excel('data.xlsx')
data.head()

,ID,Title,1st author,repo,year,abstract
0,1,Segmentized quarantine policy for managing a t...,"Kim, J.",arXiv,2024,"By the end of 2021, COVID-19 had spread to ove..."
1,2,AutoProteinEngine: A Large Language Model Driv...,"Liu, Y.",arXiv,2024,Protein engineering is important for biomedica...
2,3,Integration of Large Vision Language Models fo...,"Chen, Z.",arXiv,2024,Traditional natural disaster response involves...
3,4,Choice between Partial Trajectories,"Marklund, H.",arXiv,2024,As AI agents generate increasingly sophisticat...
4,5,Building Altruistic and Moral AI Agent with Br...,"Zhao, F.",arXiv,2024,"As AI closely interacts with human society, it..."


## Testing the base functionalities

Testing the OpenAI provider (with OpenAI and Gemini models):

In [6]:
# openanai_provider = OpenAIProvider(model="gpt-4o-mini")
openanai_provider = OpenAIProvider(model="gemini-1.5-flash")
question = "What is the capital of France?"
asyncio.run(openanai_provider.get_response(question))

('The capital of France is Paris.\n',
 {'input_cost': 5.25e-07, 'output_cost': 2.1e-06, 'total_cost': 2.625e-06})

Testing the Ollama provider:

In [5]:
ollama_provider = OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434")
question = "What is the capital of France?"
asyncio.run(ollama_provider.get_response(question))

('The capital of France is Paris.',
 {'input_cost': 0, 'output_cost': 0, 'total_cost': 0})

Testing the LiteLLM provider:

In [4]:
# litellm_provider = LiteLLMProvider(model="gpt-4o-mini")
# litellm_provider = LiteLLMProvider(model="claude-3-5-sonnet-20240620")
# litellm_provider = LiteLLMProvider(model="groq/llama-3.3-70b-versatile")
litellm_provider = LiteLLMProvider(model="groq/llama-3.3-70b-versatile")

question = "What is the capital of France?"
asyncio.run(litellm_provider.get_response(question))

response: <Response [200 OK]>


('The capital of France is Paris.', 3.464e-05)

Testing the ScoringReviewer agent:

In [34]:
agent = ScoringReviewer(
    # provider=OpenAIProvider(model="gpt-4o-mini"),
    # provider=OpenAIProvider(model="gemini-1.5-flash"),
    # provider=OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434"),
    # provider=LiteLLMProvider(model="gpt-4o-mini"),
    # provider=LiteLLMProvider(model="claude-3-5-sonnet-20240620"),
    # provider=LiteLLMProvider(model="groq/llama-3.3-70b-versatile"),
    provider=LiteLLMProvider(model="gemini/gemini-1.5-flash"),
    name="Pouria",
    max_concurrent_requests=1, 
    backstory="an expert reviewer and researcher!",
    input_description = "article title",
    model_args={"max_tokens": 100, "temperature": 0.1},
    reasoning = "brief",
    review_criteria="Look for articles that certainly do not employ any AI or machine learning agents",
    score_set=[1, 2],
    scoring_rules='Score 1 if the paper does not meet the criteria, and 2 if the paper meets the criteria.',
)


# Dummy input
text_list = data.Title.str.lower().tolist()
print("Inputs:\n\n", '\n'.join(text_list[:3]))

# Dummy review
results, total_cost = asyncio.run(agent.review_items(text_list[:3]))
print("\n\n Outputs:")
for result in results:
    print(result)

# Dummy costs
print("\nCosts:\n")
for item in agent.memory:
    print(item['cost'])

print("\nTotal cost:\n")
print(total_cost)

Inputs:

 segmentized quarantine policy for managing a tradeoff between containment of infectious disease and social cost of quarantine
autoproteinengine: a large language model driven agent framework for multimodal automl in protein engineering
integration of large vision language models for efficient post-disaster damage assessment and reporting


Reviewing 3 items - 2024-12-14 20:02:35: 100%|██████████| 3/3 [00:01<00:00,  1.58it/s]



 Outputs:
{'reasoning': 'The article title is relevant and specific, meeting the criteria.', 'score': 2}
{'reasoning': 'The article title is relevant and meets the criteria.', 'score': 2}
{'reasoning': 'The article title is relevant to post-disaster damage assessment and reporting, thus meeting the criteria.', 'score': 2}

Costs:

2.13e-05
2.0549999999999998e-05
2.3024999999999997e-05

Total cost:

2.3024999999999997e-05


## Testing the main Functionalities

#### A multiagent review workflow for doing title/abstract analysis

Setting up the agents:

In [38]:
pouria = ScoringReviewer(
    # provider=OpenAIProvider(model="gemini-1.5-flash"),
    # provider=OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434"),
    # provider=LiteLLMProvider(model="groq/llama-3.3-70b-versatile"),
    # provider=LiteLLMProvider(model="groq/llama-3.3-70b-versatile"),
    provider=LiteLLMProvider(model="gemini/gemini-1.5-flash"),
    name="Pouria",
    max_concurrent_requests=20, 
    backstory="a radiologist with many years of background in statistcis and data science, who are famous among your colleagues for your systematic thinking, organizaton of thoughts, and being conservative",
    model_args={"max_tokens": 100, "temperature": 0.1},
    input_description = "tilte and abstract of scientific articles",
    reasoning = "cot",
    scoring_task="Look for articles that disucss large languange models-based AI agents applied to medical imaging data",
    score_set=[1, 2],
    scoring_rules='Score 1 if the paper meets the criteria, and 2 if the paper does not meet the criteria.',
)

bardia = ScoringReviewer(
    provider=OpenAIProvider(model="gpt-4o-mini"),
    name="Bardia",
    max_concurrent_requests=20, 
    backstory="an expert in data science with a background in developing ML models for healthcare, who are famous among your colleagues for your creativity and out of the box thinking",
    model_args={"max_tokens": 100, "temperature": 0.8},
    input_description = "tilte and abstract of scientific articles",
    reasoning = "brief",
    scoring_task="Look for articles that disucss large languange models-based AI agents applied to medical imaging data",
    score_set=[1, 2],
    scoring_rules='Score 1 if the paper meets the criteria, and 2 if the paper does not meet the criteria.',
)

brad = ScoringReviewer(
    provider=OpenAIProvider(model="gpt-4o"),
    name="Brad",
    max_concurrent_requests=20, 
    backstory="a senior radiologist with a PhD in computer science and years of experience as the director of a DL lab focused on developing ML models for radiology and healthcare",
    input_description = "tilte and abstract of scientific articles",
    temperature=0.4,
    reasoning = "cot",
    max_tokens=100,
    scoring_task="""Pouria and Bardia have Looked for articles that disucss large languange models-based AI agents applied to medical imaging data. 
                       They scored an article 1 if they thought it does not meet this criteria, 2 if they thought it meets the criteria, 0 if they were uncertain of scoring.
                       You will receive an article they have had different opinions about, as well as each of their scores and their reasoning for that score. Read their reviews and determine who you agree with. 
                    """,
    score_set=[1, 2],
    scoring_rules="""Score 1 if you agree with Pouria, and score 2 if you agree with Bardia.""",
)


Setting up the review workflow:

In [39]:
title_abs_review = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [pouria, bardia],
            "inputs": ["Title", "abstract"]
        },
        {
            "round": 'B',
            "reviewers": [brad],
            "inputs": ["Title", "abstract", "round-A_Pouria_output", "round-A_Bardia_output"],
            "filter": lambda row: row["round-A_Pouria_output"]["score"] != row["round-A_Bardia_output"]["score"]
        }
    ]
)

Applying the review workflow to a number of sample articles:

In [41]:
# Reload the data if needed.
sample_data = pd.read_excel('data.xlsx').sample(10).reset_index(drop=True)
updated_data = asyncio.run(title_abs_review(sample_data))

print("Total cost: ")
print(title_abs_review.get_total_cost())

print("\nDetailed cost:")
print(title_abs_review.reviewer_costs)

updated_data


Starting review round A (1/2)...
Processing 10 eligible rows


['round: A', 'reviewer_name: Pouria'] - 2024-12-14 20:04:00: 100%|██████████| 10/10 [00:01<00:00,  8.81it/s]
['round: A', 'reviewer_name: Bardia'] - 2024-12-14 20:04:01: 100%|██████████| 10/10 [00:01<00:00,  7.58it/s]


Starting review round B (2/2)...
Skipping review round B - no eligible rows
Total cost: 
0.000183375

Detailed cost:
{('A', 'Pouria'): 7.3425e-05, ('A', 'Bardia'): 0.00010995}


,ID,Title,1st author,repo,year,abstract,round-A_Pouria_output,round-A_Pouria_score,round-A_Pouria_reasoning,round-A_Bardia_output,round-A_Bardia_score,round-A_Bardia_reasoning
0,174,Semantic Information in MC: Chemotaxis Beyond ...,"Brand, L.",arXiv,2024,The recently emerged molecular communication (...,{'reasoning': 'This article focuses on molecul...,2,This article focuses on molecular communicatio...,{'reasoning': 'The paper discusses molecular c...,2,The paper discusses molecular communication (M...
1,389,Multi-Pose Fusion for Sparse-View CT Reconstru...,"Yang, D.",arXiv,2022,CT imaging works by reconstructing an object o...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...,{'reasoning': 'The article discusses a novel a...,2,The article discusses a novel algorithm for CT...
2,199,Norm Enforcement with a Soft Touch: Faster Eme...,"Tzeng, S.-T.",arXiv,2024,A multiagent system is a society of autonomous...,{'reasoning': 'The provided abstract does not ...,2,The provided abstract does not discuss large l...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...
3,345,Arena-Web - A Web-based Development and Benchm...,"Kästner, L.",arXiv,2023,"In recent years, mobile robot navigation appro...",{'reasoning': 'The abstract does not mention l...,2,The abstract does not mention large language m...,{'reasoning': 'The article discusses a platfor...,2,The article discusses a platform for robot nav...
4,524,Learning from zero: how to make consumption-sa...,"Shi, R.",arXiv,2021,This exercise proposes a learning mechanism to...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...,{'reasoning': 'The article discusses decision-...,2,The article discusses decision-making in econo...
5,616,Impact of tumor-parenchyma biomechanics on liv...,"Wang, Y.",bioRxiv,2020,Colorectal carcinoma (CRC) and other cancers o...,{'reasoning': 'The article focuses on a multi-...,2,The article focuses on a multi-model approach ...,{'reasoning': 'The article focuses on biomecha...,2,The article focuses on biomechanical interacti...
6,79,PersLLM: A Personified Training Approach for L...,"Zeng, Z.",arXiv,2024,Large language models (LLMs) demonstrate human...,{'reasoning': 'The abstract does not mention m...,2,The abstract does not mention medical imaging ...,{'reasoning': 'The article discusses personali...,2,The article discusses personality development ...
7,239,A MULTI-AGENT REINFORCEMENT LEARNING FRAMEWORK...,"Sharma, D.",arXiv,2023,Human immunodeficiency virus (HIV) is a major ...,{'reasoning': 'The article focuses on a multi-...,2,The article focuses on a multi-agent reinforce...,{'reasoning': 'The article discusses a multi-a...,2,The article discusses a multi-agent reinforcem...
8,426,Human-in-the-loop online multi-agent approach ...,"Bravo-Rocca, G.",arXiv,2022,"Increasing a ML model accuracy is not enough, ...",{'reasoning': 'The abstract does not mention t...,2,The abstract does not mention the application ...,{'reasoning': 'The article discusses a multi-a...,2,The article discusses a multi-agent approach t...
9,160,Modeling the Spread of COVID-19 in University ...,"Herrmann, J.W.",arXiv,2024,Mathematical and simulation models are often u...,{'reasoning': 'The article does not discuss la...,2,The article does not discuss large language mo...,{'reasoning': 'The article discusses mathemati...,2,The article discusses mathematical modeling of...


In [42]:
for i, row in updated_data.iterrows():
    print(
        f"""
        Title: {row.Title}
        Abstract: {row.abstract}
        Pouria's score: {row["round-A_Pouria_score"]}
        Pouria's reasoning: {row["round-A_Pouria_reasoning"]}
        Bardia's score: {row["round-A_Bardia_score"]}
        Bardia's reasoning: {row["round-A_Bardia_reasoning"]}
        Brad's score: {None if "round-B_Brad_score" not in row else row["round-B_Brad_score"]}
        Brad's reasoning: {None if "round-B_Brad_reasoning" not in row else row["round-B_Brad_reasoning"]}
        """
    )


        Title: Semantic Information in MC: Chemotaxis Beyond Shannon
        Abstract: The recently emerged molecular communication (MC) paradigm intends to leverage communication engineering tools for the design of synthetic chemical communication systems. These systems are envisioned to operate at nanoscale and in biological environments, such as the human body, and catalyze the emergence of revolutionary applications in the context of early disease monitoring and drug targeting. Despite the abundance of theoretical (and recently also experimental) MC system designs proposed over the past years, some fundamental questions remain unresolved, hindering the breakthrough of MC in real-world applications. One of these questions is: What can be a useful measure of information in the context of MC applications? While most existing works on MC build upon the concept of syntactic information as introduced by Shannon, in this paper, we explore the framework of semantic information as introduc